In [315]:
import numpy as np
import pandas as pd

import pickle as pkl

import gc
from sklearn.model_selection import KFold
from cuml.metrics import mean_absolute_error, mean_squared_error

import seaborn as sns

In [316]:
out_base_path = '/media/hiroki/working/kaggle/trends-neuroimaging/models'

In [317]:
OUTPUT_DICT = ''

ID = 'Id'
TARGET_COLS = ['age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2']
SEED = 2020

N_FOLD = 15

In [318]:
base_path = '/media/hiroki/share/kaggle_data/trends-assessment-prediction/'
train = pd.read_csv(base_path+'train_scores.csv', dtype={'Id':str})\
            #.dropna().reset_index(drop=True) # to make things easy
reveal_ID = pd.read_csv(base_path+'reveal_ID_site2.csv', dtype={'Id':str})
ICN_numbers = pd.read_csv(base_path+'ICN_numbers.csv')
loading = pd.read_csv(base_path+'loading.csv', dtype={'Id':str})
fnc = pd.read_csv(base_path+'fnc.csv', dtype={'Id':str})
sample_submission = pd.read_csv(base_path+'sample_submission.csv', dtype={'Id':str})

In [319]:
fnc_features, loading_features = list(fnc.columns[1:]), list(loading.columns[1:])

In [320]:
sample_submission['ID_num'] = sample_submission[ID].apply(lambda x: int(x.split('_')[0]))
test = pd.DataFrame({ID: sample_submission['ID_num'].unique().astype(str)})
del sample_submission['ID_num']; gc.collect()
test.head()

,Id
0,10003
1,10006
2,10010
3,10011
4,10012


In [321]:
test.head()

,Id
0,10003
1,10006
2,10010
3,10011
4,10012


In [322]:
# merge
train = train.merge(loading, on=ID, how='left')
train = train.merge(fnc, on=ID, how='left')
train

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2,IC_01,IC_07,IC_05,IC_16,...,CBN(13)_vs_DMN(94),CBN(18)_vs_DMN(94),CBN(4)_vs_DMN(94),CBN(7)_vs_DMN(94),CBN(18)_vs_CBN(13),CBN(4)_vs_CBN(13),CBN(7)_vs_CBN(13),CBN(4)_vs_CBN(18),CBN(7)_vs_CBN(18),CBN(7)_vs_CBN(4)
0,10001,57.436077,30.571975,62.553736,53.325130,51.427998,0.006070,0.014466,0.004136,0.000658,...,-0.149279,0.552841,0.131046,0.335446,0.394867,-0.042853,0.124627,-0.060712,0.515964,0.290488
1,10002,59.580851,50.969456,67.470628,60.651856,58.311361,0.009087,0.009291,0.007049,-0.002076,...,-0.214216,-0.039792,0.143014,-0.189962,0.498373,0.444231,0.592438,0.028649,0.705524,0.248327
2,10004,71.413018,53.152498,58.012103,52.418389,62.536641,0.004675,0.000957,0.006154,-0.000429,...,-0.130339,0.309540,0.141469,0.030853,0.344394,0.214097,0.317556,0.012435,0.665937,0.081358
3,10005,66.532630,NaN,NaN,52.108977,69.993075,-0.000398,0.006878,0.009051,0.000369,...,-0.139525,0.394932,0.040443,0.428334,0.498837,0.266755,0.227379,0.028984,0.752343,0.087898
4,10007,38.617381,49.197021,65.674285,40.151376,34.096421,0.005192,0.010585,0.012160,-0.000920,...,-0.150218,0.408926,0.072004,0.157582,0.532046,0.355448,0.462675,0.161005,0.703679,0.293607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5872,21746,14.257265,21.358872,61.165998,51.778483,54.640179,-0.001115,0.007108,0.008652,0.003596,...,-0.249481,0.205351,0.012067,0.310750,0.426335,0.193812,0.158720,0.055471,0.568766,0.160516
5873,21747,55.456978,68.169675,29.907995,55.349257,54.019517,0.007263,0.016489,0.012704,0.004357,...,-0.119170,0.201846,-0.008290,0.119828,0.551936,0.598931,0.511816,0.303312,0.704483,0.461588
5874,21750,48.948756,55.114811,60.878271,38.617246,50.679885,0.005996,0.003873,0.012353,0.000242,...,-0.103786,0.375065,0.104857,0.262614,0.502715,0.322353,0.458041,0.343754,0.705207,0.341224
5875,21752,66.532630,59.844808,72.303110,55.458281,46.870235,0.000627,0.011407,0.010957,0.000534,...,0.229712,0.431489,0.039062,0.119474,0.523894,0.445209,0.332011,0.228977,0.560968,0.263504


In [323]:
# merge
test = test.merge(loading, on=ID, how='left')
test = test.merge(fnc, on=ID, how='left')
test

,Id,IC_01,IC_07,IC_05,IC_16,IC_26,IC_06,IC_10,IC_09,IC_18,...,CBN(13)_vs_DMN(94),CBN(18)_vs_DMN(94),CBN(4)_vs_DMN(94),CBN(7)_vs_DMN(94),CBN(18)_vs_CBN(13),CBN(4)_vs_CBN(13),CBN(7)_vs_CBN(13),CBN(4)_vs_CBN(18),CBN(7)_vs_CBN(18),CBN(7)_vs_CBN(4)
0,10003,0.008151,0.014684,0.010444,-0.005293,-0.002913,0.015042,0.017745,0.003930,-0.008021,...,-0.154941,0.136850,-0.022361,0.137625,0.677972,0.409412,0.563892,0.438684,0.618204,0.284474
1,10006,0.000334,0.005311,0.010053,0.006920,-0.000065,0.015310,0.016543,0.004794,0.003982,...,-0.053606,0.240957,0.270419,0.367692,0.354501,0.486364,0.416908,0.285274,0.693490,0.448526
2,10010,0.007103,0.006144,0.009770,-0.002884,-0.001346,0.015651,0.011613,-0.003291,0.013423,...,-0.244332,0.272077,0.193523,0.192254,0.563982,0.124482,0.488926,0.083368,0.774299,0.129327
3,10011,0.004362,0.010240,0.010167,0.004492,-0.001623,0.017381,0.014680,0.007453,0.008786,...,-0.099726,0.557121,0.042626,0.179456,0.416546,0.445402,0.436909,0.165182,0.591561,0.306678
4,10012,-0.007521,-0.003918,0.008434,-0.001145,0.002017,0.015065,0.019616,0.004140,-0.003744,...,-0.025230,0.203298,0.173427,0.046047,0.561599,0.418268,0.609517,0.218285,0.790285,0.301010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5872,21745,0.005406,0.006275,0.012252,0.003518,0.001400,0.015054,0.015373,0.001532,0.003546,...,0.179080,0.580813,0.202241,0.254870,0.458581,0.434638,0.587167,0.009854,0.472956,0.342085
5873,21748,0.004240,0.009213,0.010981,0.000443,-0.003072,0.010702,0.014673,0.005523,0.005780,...,-0.106345,0.234340,0.138985,0.343382,0.708744,0.312812,0.536501,0.214803,0.849512,0.204741
5874,21749,0.004783,0.017910,0.012128,-0.005683,-0.011613,0.017000,0.007230,0.001315,0.008788,...,-0.165575,0.170154,-0.029638,0.383761,0.398305,0.578621,0.357127,0.009479,0.609545,0.317230
5875,21751,0.003835,0.015067,0.015428,-0.002030,0.001205,0.012396,0.011026,-0.001491,0.005310,...,-0.087604,0.131902,-0.047932,0.022317,0.583869,0.596734,0.515209,0.379589,0.568422,0.439016


In [324]:
import IPython

def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

In [325]:
def metric(y_true, y_pred):
    return np.mean(np.sum(np.abs(y_true - y_pred), axis=0)/np.sum(y_true, axis=0))

In [326]:
# Giving less importance to FNC features since they are easier to overfit due to high dimensionality.
FNC_SCALE = 1/600

train[fnc_features] *= FNC_SCALE
test[fnc_features] *= FNC_SCALE

In [327]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(train_x, train_y, test_x, model, target_name):
    preds = []
    preds_test = []
    va_idxes = []
    
    score = []
    mae = []
    rmse = []

    # shuffleしなくても良い
    kf = KFold(n_splits=N_FOLD, shuffle=True, random_state=SEED)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx].values, train_x.iloc[va_idx].values
        tr_y, va_y = train_y.iloc[tr_idx].values, train_y.iloc[va_idx].values
        model.fit(tr_x, tr_y)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)
        
        score.append(metric(va_y, pred))
        mae.append(mean_absolute_error(va_y, pred))
        rmse.append(np.sqrt(mean_squared_error(va_y, pred)))
        
    score_cv = np.array(score).mean()
    mae_cv = np.array(mae).mean()
    rmse_cv = np.array(rmse).mean()
    print("{0}_score:{1}".format(target_name, np.round(score_cv, 8)))
    print("{0}_mae:{1}".format(target_name, np.array(mae_cv).mean()))
    print("{0}_rmse:{1}".format(target_name, np.array(rmse_cv).mean()))
    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test, score_cv

# 1st layer

## SVR

In [328]:
import cudf
import cupy as cp
from cuml import SVR

In [329]:
with open(out_base_path + '/1st_layer/svm/svm_train.pkl', 'rb') as f:
  pred_train_targets_svr_dfs = pkl.load(f)

with open(out_base_path + '/1st_layer/svm/svm_test.pkl', 'rb') as f:
  pred_test_targets_svr_dfs = pkl.load(f)

In [330]:
for i, pred_df in enumerate(pred_train_targets_svr_dfs.values()):
    if i == 0:
        pred_train_dfs = pred_df
        #display(pred_train_dfs, head=False)
    else:
        pred_train_dfs = pd.concat([pred_train_dfs, pred_df.drop("Id", axis=1)], axis=1)
        #display(pred_train_dfs, head=False)

In [331]:
for i, pred_df in enumerate(pred_test_targets_svr_dfs.values()):
    if i == 0:
        pred_test_dfs = pred_df
        #display(pred_test_dfs, head=False)
    else:
        pred_test_dfs = pd.concat([pred_test_dfs, pred_df.drop("Id", axis=1)], axis=1)
        #display(pred_test_dfs, head=False)

## baysian ridge

In [332]:
with open(out_base_path + '/1st_layer/bayes_ridge/bayes_ridge_train.pkl', 'rb') as f:
  pred_train_targets_bayes_ridge_dfs = pkl.load(f)

with open(out_base_path + '/1st_layer/bayes_ridge/bayes_ridge_test.pkl', 'rb') as f:
  pred_test_targets_bayes_ridge_dfs = pkl.load(f)

In [333]:
for i, pred_df in enumerate(pred_train_targets_bayes_ridge_dfs.values()):
    pred_train_dfs = pd.concat([pred_train_dfs, pred_df.drop("Id", axis=1)], axis=1)

for i, pred_df in enumerate(pred_test_targets_bayes_ridge_dfs.values()):
    pred_test_dfs = pd.concat([pred_test_dfs, pred_df.drop("Id", axis=1)], axis=1)

In [364]:
loading.shape

(11754, 27)

In [334]:
#with open(out_base_path + '/1st_layer/bayes_ridge_pc_ica/bayes_ridge_train.pkl', 'rb') as f:
#  pred_train_targets_bayes_ridge_dfs_PC_IC = pkl.load(f)

#with open(out_base_path + '/1st_layer/bayes_ridge_pc_ica/bayes_ridge_test.pkl', 'rb') as f:
#  pred_test_targets_bayes_ridge_dfs_PC_IC = pkl.load(f)

In [335]:
#for i, pred_df in enumerate(pred_train_targets_bayes_ridge_dfs_PC_IC.values()):
#    pred_train_dfs = pd.concat([pred_train_dfs, pred_df.drop("Id", axis=1)], axis=1)
    
#for i, pred_df in enumerate(pred_test_targets_bayes_ridge_dfs_PC_IC.values()):
#    pred_test_dfs = pd.concat([pred_test_dfs, pred_df.drop("Id", axis=1)], axis=1)

## BaggingRegressor

In [336]:
with open(out_base_path + '/1st_layer/bagging_ridge/bagging_ridge_train.pkl', 'rb') as f:
  pred_train_targets_bagging_ridge_dfs = pkl.load(f)

with open(out_base_path + '/1st_layer/bagging_ridge/bagging_ridge_test.pkl', 'rb') as f:
  pred_test_targets_bagging_ridge_dfs = pkl.load(f)

In [337]:
for i, pred_df in enumerate(pred_train_targets_bagging_ridge_dfs.values()):
    pred_train_dfs = pd.concat([pred_train_dfs, pred_df.drop("Id", axis=1)], axis=1)

for i, pred_df in enumerate(pred_test_targets_bagging_ridge_dfs.values()):
    pred_test_dfs = pd.concat([pred_test_dfs, pred_df.drop("Id", axis=1)], axis=1)

In [338]:
#with open(out_base_path + '/1st_layer/bagging_ridge_pc_ica/bagging_ridge_train.pkl', 'rb') as f:
#  pred_train_targets_bagging_ridge_dfs_PC_IC = pkl.load(f)

#with open(out_base_path + '/1st_layer/bagging_ridge_pc_ica/bagging_ridge_test.pkl', 'rb') as f:
#  pred_test_targets_bagging_ridge_dfs_PC_IC = pkl.load(f)

In [339]:
#for i, pred_df in enumerate(pred_train_targets_bagging_ridge_dfs_PC_IC.values()):
#    pred_train_dfs = pd.concat([pred_train_dfs, pred_df.drop("Id", axis=1)], axis=1)
#for i, pred_df in enumerate(pred_test_targets_bagging_ridge_dfs_PC_IC.values()):
#    pred_test_dfs = pd.concat([pred_test_dfs, pred_df.drop("Id", axis=1)], axis=1)

# LightGBM

In [340]:
with open(out_base_path + '/1st_layer/lgb_pc_and_raw/lgb_train.pkl', 'rb') as f:
  pred_train_targets_lgb_dfs = pkl.load(f)

with open(out_base_path + '/1st_layer/lgb_pc_and_raw/lgb_test.pkl', 'rb') as f:
  pred_test_targets_lgb_dfs = pkl.load(f)

In [341]:
for i, pred_df in enumerate(pred_train_targets_lgb_dfs.values()):
    pred_train_dfs = pd.concat([pred_train_dfs, pred_df.drop("Id", axis=1)], axis=1)

for i, pred_df in enumerate(pred_test_targets_lgb_dfs.values()):
    pred_test_dfs = pd.concat([pred_test_dfs, pred_df.drop("Id", axis=1)], axis=1)

In [342]:
with open(out_base_path + '/1st_layer/lgb_fncPC_and_loadingIC_and_raw/lgb_train.pkl', 'rb') as f:
  pred_train_targets_lgb_dfs_PC_IC = pkl.load(f)

with open(out_base_path + '/1st_layer/lgb_fncPC_and_loadingIC_and_raw/lgb_test.pkl', 'rb') as f:
  pred_test_targets_lgb_dfs_PC_IC = pkl.load(f)

In [343]:
for i, pred_df in enumerate(pred_train_targets_lgb_dfs_PC_IC.values()):
    pred_train_dfs = pd.concat([pred_train_dfs, pred_df.drop("Id", axis=1)], axis=1)

for i, pred_df in enumerate(pred_test_targets_lgb_dfs_PC_IC.values()):
    pred_test_dfs = pd.concat([pred_test_dfs, pred_df.drop("Id", axis=1)], axis=1)

# 2nd layer

In [362]:
pred_train_dfs

,Id,pre_svr_age,pre_svr_domain1_var1,pre_svr_domain1_var2,pre_svr_domain2_var1,pre_svr_domain2_var2,pre_bayRidge_age,pre_bayRidge_domain1_var1,pre_bayRidge_domain1_var2,pre_bayRidge_domain2_var1,...,pre_lgb_PCICraw_age,pre_lgb_PCICraw_domain1_var1,pre_lgb_PCICraw_domain1_var2,pre_lgb_PCICraw_domain2_var1,pre_lgb_PCICraw_domain2_var2,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
0,10001,60.877768,53.299463,59.728622,49.506853,50.245806,60.906731,53.632844,58.933031,48.193288,...,61.186209,55.115084,58.697368,48.847452,51.115512,57.436077,30.571975,62.553736,53.325130,51.427998
1,10002,55.056918,54.708045,59.462757,48.809679,52.669451,56.150469,53.087283,59.347237,47.758491,...,56.481601,54.056957,57.309726,45.576009,52.374991,59.580851,50.969456,67.470628,60.651856,58.311361
2,10004,51.727589,51.827130,60.177994,47.702372,50.088077,52.027515,52.085337,58.739435,46.646664,...,52.497028,52.347872,59.157301,46.650965,49.776307,71.413018,53.152498,58.012103,52.418389,62.536641
3,10005,61.276086,NaN,NaN,50.182354,54.036360,60.813771,NaN,NaN,48.631294,...,56.492725,NaN,NaN,48.256743,52.621749,66.532630,NaN,NaN,52.108977,69.993075
4,10007,44.009071,49.981281,58.200460,46.080720,52.361837,43.660350,49.480579,59.537922,46.166929,...,42.496281,50.131152,56.728271,46.172037,52.447741,38.617381,49.197021,65.674285,40.151376,34.096421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5872,21746,33.566653,50.700319,60.844463,45.499232,50.331900,34.199012,50.407656,59.896793,45.761590,...,36.091235,48.128237,58.956236,45.009958,49.747162,14.257265,21.358872,61.165998,51.778483,54.640179
5873,21747,51.158271,48.903705,58.519688,47.681367,54.147085,51.591765,48.533070,58.785206,48.017085,...,48.702672,47.803205,58.452526,46.032770,52.196401,55.456978,68.169675,29.907995,55.349257,54.019517
5874,21750,40.541940,49.380542,62.664363,46.724739,53.457566,41.323194,49.492569,59.906589,46.374749,...,43.025708,50.436384,60.623237,46.563148,52.896559,48.948756,55.114811,60.878271,38.617246,50.679885
5875,21752,55.981565,54.848841,58.994825,48.854849,54.597196,55.273854,55.548991,59.230211,48.076530,...,55.093901,54.320712,58.881641,45.125422,51.773267,66.532630,59.844808,72.303110,55.458281,46.870235


In [363]:
pred_test_dfs

,Id,pre_svr_age,pre_svr_domain1_var1,pre_svr_domain1_var2,pre_svr_domain2_var1,pre_svr_domain2_var2,pre_bayRidge_age,pre_bayRidge_domain1_var1,pre_bayRidge_domain1_var2,pre_bayRidge_domain2_var1,...,pre_lgb_age,pre_lgb_domain1_var1,pre_lgb_domain1_var2,pre_lgb_domain2_var1,pre_lgb_domain2_var2,pre_lgb_PCICraw_age,pre_lgb_PCICraw_domain1_var1,pre_lgb_PCICraw_domain1_var2,pre_lgb_PCICraw_domain2_var1,pre_lgb_PCICraw_domain2_var2
0,10003,51.174243,50.344552,61.387124,47.831571,58.071251,57.277173,48.902719,59.604744,49.166154,...,54.292336,50.079941,58.403966,47.962857,54.320618,51.381175,50.019790,58.340222,48.587263,54.433086
1,10006,62.799943,54.825729,59.661494,50.494938,51.920740,62.225247,54.251414,58.417728,49.379944,...,56.959581,53.211325,59.347404,47.537110,51.488241,58.519583,52.619184,58.636491,47.684004,51.783328
2,10010,40.416030,50.427917,60.350332,45.898079,53.258320,37.151168,50.050625,59.449547,45.502617,...,36.468852,47.877139,59.909216,45.575182,52.491431,37.624508,48.807663,59.705971,45.636117,52.209902
3,10011,49.488949,52.684360,60.442198,47.899687,51.495168,51.249614,52.089598,59.673240,47.195586,...,47.100583,52.052349,60.031853,48.772689,50.414979,47.653619,52.352747,59.923230,48.338063,50.702555
4,10012,51.861948,55.367383,57.786364,45.597591,58.524878,52.133715,55.197381,57.916459,46.060805,...,57.226663,54.606112,55.939950,47.986137,55.271238,57.012916,54.723552,56.038921,47.293969,55.024506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5872,21745,39.215215,50.607787,58.170242,46.252299,54.214536,39.921680,50.774636,59.133815,45.692795,...,42.341586,51.710411,58.212820,46.493303,53.302224,45.236659,52.346856,58.373315,46.383479,52.848224
5873,21748,57.492252,57.586491,60.148703,50.269298,53.274539,54.204070,57.345042,59.246141,48.603110,...,53.474764,54.163639,60.062991,47.069057,52.768913,54.824890,54.127723,59.696752,46.873795,52.600070
5874,21749,42.109480,47.428567,60.133285,45.114528,51.289126,40.112965,46.884474,59.818693,45.974274,...,39.593692,47.299449,59.213545,44.957938,51.023382,41.670373,48.610427,58.839367,45.438216,51.058686
5875,21751,39.902905,51.145214,58.695608,46.097978,56.901258,40.363149,50.894871,59.312741,45.832426,...,40.673278,50.076689,57.815019,46.121214,54.151109,45.632117,51.341288,58.342882,46.169253,53.890403


In [344]:
pred_train_dfs = pd.concat([pred_train_dfs, train[TARGET_COLS]], axis=1)

In [356]:
pred_train_dfs.shape

(5877, 31)

In [357]:
target_columns = pred_test_dfs.loc[
        :, pred_test_dfs.columns.str.contains('age')
    ].columns

In [360]:
pred_train_dfs[target_columns].describe()

,pre_svr_age,pre_bayRidge_age,pre_bagRidge_age,pre_lgb_age,pre_lgb_PCICraw_age
count,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000
mean,50.196965,50.038717,50.019740,50.019480,50.029789
std,9.773460,10.038128,7.129463,8.951893,8.789135
min,20.334450,19.518356,25.687146,24.319649,25.321898
25%,43.038603,42.842211,44.908780,43.476473,43.532856
50%,49.934504,49.803500,49.831197,49.774657,49.709806
75%,57.185065,56.988511,54.942511,56.380226,56.322313
max,87.422299,83.939578,74.476890,76.546682,77.848962


In [359]:
pred_test_dfs[target_columns].describe()

,pre_svr_age,pre_bayRidge_age,pre_bagRidge_age,pre_lgb_age,pre_lgb_PCICraw_age
count,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000
mean,49.106156,49.101251,49.279251,49.022665,49.483919
std,9.915822,10.253920,7.232693,8.951931,8.749833
min,18.709324,16.523534,26.621817,24.844154,25.521332
25%,41.918888,41.717775,44.098387,42.208266,42.913910
50%,48.801923,48.786756,49.241908,48.615730,49.258425
75%,56.206817,56.369883,54.412898,55.622577,55.767654
max,81.962472,83.799690,73.409756,76.678734,76.634562


In [349]:
pred_train_dfs.shape

(5877, 31)

In [350]:
pred_test_dfs.shape

(5877, 26)

In [351]:
pred_train_dfs

,Id,pre_svr_age,pre_svr_domain1_var1,pre_svr_domain1_var2,pre_svr_domain2_var1,pre_svr_domain2_var2,pre_bayRidge_age,pre_bayRidge_domain1_var1,pre_bayRidge_domain1_var2,pre_bayRidge_domain2_var1,...,pre_lgb_PCICraw_age,pre_lgb_PCICraw_domain1_var1,pre_lgb_PCICraw_domain1_var2,pre_lgb_PCICraw_domain2_var1,pre_lgb_PCICraw_domain2_var2,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
0,10001,60.877768,53.299463,59.728622,49.506853,50.245806,60.906731,53.632844,58.933031,48.193288,...,61.186209,55.115084,58.697368,48.847452,51.115512,57.436077,30.571975,62.553736,53.325130,51.427998
1,10002,55.056918,54.708045,59.462757,48.809679,52.669451,56.150469,53.087283,59.347237,47.758491,...,56.481601,54.056957,57.309726,45.576009,52.374991,59.580851,50.969456,67.470628,60.651856,58.311361
2,10004,51.727589,51.827130,60.177994,47.702372,50.088077,52.027515,52.085337,58.739435,46.646664,...,52.497028,52.347872,59.157301,46.650965,49.776307,71.413018,53.152498,58.012103,52.418389,62.536641
3,10005,61.276086,NaN,NaN,50.182354,54.036360,60.813771,NaN,NaN,48.631294,...,56.492725,NaN,NaN,48.256743,52.621749,66.532630,NaN,NaN,52.108977,69.993075
4,10007,44.009071,49.981281,58.200460,46.080720,52.361837,43.660350,49.480579,59.537922,46.166929,...,42.496281,50.131152,56.728271,46.172037,52.447741,38.617381,49.197021,65.674285,40.151376,34.096421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5872,21746,33.566653,50.700319,60.844463,45.499232,50.331900,34.199012,50.407656,59.896793,45.761590,...,36.091235,48.128237,58.956236,45.009958,49.747162,14.257265,21.358872,61.165998,51.778483,54.640179
5873,21747,51.158271,48.903705,58.519688,47.681367,54.147085,51.591765,48.533070,58.785206,48.017085,...,48.702672,47.803205,58.452526,46.032770,52.196401,55.456978,68.169675,29.907995,55.349257,54.019517
5874,21750,40.541940,49.380542,62.664363,46.724739,53.457566,41.323194,49.492569,59.906589,46.374749,...,43.025708,50.436384,60.623237,46.563148,52.896559,48.948756,55.114811,60.878271,38.617246,50.679885
5875,21752,55.981565,54.848841,58.994825,48.854849,54.597196,55.273854,55.548991,59.230211,48.076530,...,55.093901,54.320712,58.881641,45.125422,51.773267,66.532630,59.844808,72.303110,55.458281,46.870235


In [352]:
from sklearn.linear_model import BayesianRidge

In [353]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [354]:
layer2_pred_train_targets_bayes_ridge = {}
layer2_pred_test_targets_bayes_ridge = {}

layer2_pred_train_targets_bayes_ridge_dfs = {}
layer2_pred_test_targets_bayes_ridge_dfs = {}

overal_score = 0

print("N_FOLD:{}".format(N_FOLD))

for target, w in [("age", 0.3),
                  ("domain1_var1", 0.175),
                  ("domain1_var2", 0.175),
                  ("domain2_var1", 0.175),
                  ("domain2_var2", 0.175)]:
    target_columns = pred_train_dfs.loc[
        :, pred_train_dfs.columns.str.contains('_'+target)
    ].columns
    train_df2 = pred_train_dfs[pred_train_dfs[target].notnull()]
    use_idx = train_df2.index
    train_x_2 = train_df2[target_columns]
    ss.fit(train_x_2)
    train_x_2 = ss.fit_transform(train_x_2)
    train_x_2 = pd.DataFrame(train_x_2)
    
    train_y = train_df2[target]
    
    test_df2 = pred_test_dfs
    test_x_2 = test_df2[target_columns]
    test_x_2 = ss.fit_transform(test_x_2)
    test_x_2 = pd.DataFrame(test_x_2)
    
    bayes_ridge = BayesianRidge(n_iter = 3000)
    
    print("-----{}-----".format(target))
    pred_train, preds_test, score_cv = predict_cv(train_x_2, train_y, test_x_2, bayes_ridge, target)
    overal_score += w*score_cv
    layer2_pred_train_targets_bayes_ridge[target] = pred_train
    layer2_pred_test_targets_bayes_ridge[target] = preds_test
    
    layer2_pred_train_targets_bayes_ridge_dfs[target] = pd.Series(pred_train, name="pre_bayRidge_2{}".format(target), index=use_idx)
    layer2_pred_train_targets_bayes_ridge_dfs[target] = pd.merge(train['Id'],
                                                 pred_train_targets_bayes_ridge_dfs[target],
                                                 left_index=True,
                                                 right_index=True)
    layer2_pred_test_targets_bayes_ridge_dfs[target] = pd.Series(preds_test, name="pre_bayRidge_2{}".format(target))
    layer2_pred_test_targets_bayes_ridge_dfs[target] = pd.concat([test['Id'], pred_test_targets_bayes_ridge_dfs[target]], axis=1)

    
    
print('--------------------------------------------')
print("Overal score:", np.round(overal_score, 8))

N_FOLD:5
-----age-----
age_score:0.14228945
age_mae:7.118897902481012
age_rmse:9.034607051984661
-----domain1_var1-----
domain1_var1_score:0.15068614
domain1_var1_mae:7.757071015680434
domain1_var1_rmse:9.664620442977448
-----domain1_var2-----
domain1_var2_score:0.15090531
domain1_var2_mae:8.939693004593186
domain1_var2_rmse:11.353839358340752
-----domain2_var1-----
domain2_var1_score:0.18161991
domain2_var1_mae:8.592875198643966
domain2_var1_rmse:10.880877815259264
-----domain2_var2-----
domain2_var2_score:0.17621198
domain2_var2_mae:9.145310597476684
domain2_var2_rmse:11.68338500268423
--------------------------------------------
Overal score: 0.15808592


In [355]:
Overal score: 0.15835412 -> Overal score: 0.15814395 -> Overal score: 0.15808592

SyntaxError: invalid syntax (<ipython-input-355-ebc192e55da5>, line 1)

In [ ]:
layer2_pred_train_targets_bayes_ridge_dfs = {}
layer2_pred_test_targets_bayes_ridge_dfs

In [125]:
layer2_pred_train_targets_bayes_ridge

{'age': array([61.88760206, 56.19549337, 52.77714042, ..., 41.33108853,
        56.11144037, 69.85771243]),
 'domain1_var1': array([55.00470674, 54.1279816 , 51.98135145, ..., 49.73022642,
        55.36993931, 59.04412051]),
 'domain1_var2': array([58.88487006, 58.36501636, 59.37989013, ..., 60.6325349 ,
        59.09344979, 59.60300272]),
 'domain2_var1': array([49.05309518, 46.56719932, 46.71607231, ..., 46.5781653 ,
        46.45228146, 50.79321437]),
 'domain2_var2': array([50.49652405, 51.74413743, 49.4236524 , ..., 52.54770101,
        51.91575933, 53.22871701])}

In [128]:
for df in layer2_pred_train_targets_bayes_ridge_dfs.values():
    display(df.describe())
#pred_train_targets_df = pd.DataFrame(layer2_pred_train_targets_bayes_ridge)
#pred_test_targets_df = pd.DataFrame(layer2_pred_test_targets_bayes_ridge)

,pre_bayRidge_age
count,5877.000000
mean,50.038717
std,10.038128
min,19.518356
25%,42.842211


,pre_bayRidge_domain1_var1
count,5439.000000
mean,51.492404
std,3.185482
min,40.796837
25%,49.327597


,pre_bayRidge_domain1_var2
count,5439.000000
mean,59.245596
std,0.645455
min,56.373948
25%,58.836050


,pre_bayRidge_domain2_var1
count,5838.000000
mean,47.321834
std,2.256074
min,40.439732
25%,45.751823


,pre_bayRidge_domain2_var2
count,5838.000000
mean,51.903249
std,1.865373
min,40.634094
25%,50.685836


In [129]:
with open (out_base_path+'/2nd_layer/bayes_ridge/ver2/bayes_ridge_train2.pkl', 'wb') as f:
  pkl.dump(layer2_pred_test_targets_bayes_ridge_dfs , f)

with open (out_base_path+'/2nd_layer/bayes_ridge/ver2/bayes_ridge_test2.pkl', 'wb') as f:
  pkl.dump(pred_test_targets_bagging_ridge_dfs , f)

# Submit

In [130]:
test_df = pd.concat([test['Id'], pred_test_targets_df], axis=1)

In [34]:
test_df

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
0,10003,56.093646,49.350655,58.896474,48.935203,56.202111
1,10006,62.356282,54.178181,58.854373,48.839883,51.706693
2,10010,38.427866,48.947544,59.538148,45.203810,52.659198
3,10011,50.593086,52.207185,59.953497,48.745056,50.734754
4,10012,53.721316,55.387162,56.712267,47.249271,57.371622
...,...,...,...,...,...,...
5872,21745,40.343817,51.049262,58.766930,46.157355,53.923683
5873,21748,56.579389,56.569498,59.864522,47.787383,52.736939
5874,21749,41.237813,46.558849,59.493404,45.198264,50.899132
5875,21751,40.809825,50.443889,58.741290,45.507226,55.808851


In [68]:
test_df

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
0,10003,55.738271,49.272082,58.923767,48.935058,56.170406
1,10006,63.345616,54.309045,58.703233,48.824776,51.730516
2,10010,39.413663,49.760247,59.352942,45.258089,52.595287
3,10011,51.463856,52.165475,59.870948,48.533458,50.852033
4,10012,52.983837,55.607438,56.904215,47.033798,57.313630
...,...,...,...,...,...,...
5872,21745,40.615615,51.048883,58.895316,46.151702,53.875924
5873,21748,56.423210,56.542430,59.787313,47.663472,52.674281
5874,21749,41.860067,46.538843,59.483453,45.494613,50.876778
5875,21751,41.134915,50.540838,58.966287,45.408914,55.829795


In [35]:
test_df.describe()

,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
count,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000
mean,50.034298,51.473980,59.243808,47.328513,51.904839
std,10.092185,3.224970,0.772461,2.355579,1.954457
min,19.843688,40.516143,56.129256,39.381267,44.471556
25%,42.599854,49.206657,58.714602,45.691519,50.550907
50%,49.608588,51.492661,59.264457,47.302188,51.852833
75%,57.244846,53.745135,59.775948,48.992372,53.156977
max,82.838858,62.436266,61.681675,55.561685,60.247357


In [132]:
test_df.to_csv('/media/hiroki/working/kaggle/trends-neuroimaging/models/submit/stacking_pre_test_df2.csv')

In [133]:
sub_df = pd.melt(test_df[["Id", "age", "domain1_var1", "domain1_var2", "domain2_var1", "domain2_var2"]], id_vars=["Id"], value_name="Predicted")
sub_df["Id"] = sub_df["Id"].astype("str") + "_" +  sub_df["variable"].astype("str")

sub_df = sub_df.drop("variable", axis=1).sort_values("Id")
assert sub_df.shape[0] == test_df.shape[0]*5
sub_df.head(10)

,Id,Predicted
0,10003_age,55.479157
5877,10003_domain1_var1,49.318236
11754,10003_domain1_var2,58.770805
17631,10003_domain2_var1,49.184022
23508,10003_domain2_var2,56.308094
1,10006_age,62.333648
5878,10006_domain1_var1,53.996464
11755,10006_domain1_var2,58.721850
17632,10006_domain2_var1,48.877938
23509,10006_domain2_var2,51.808687


In [134]:
sub_df.to_csv('/media/hiroki/working/kaggle/trends-neuroimaging/models/submit/submit_pcic.csv', index=False)